Quantum Operation Quantum Operation  
Yes we use [reduplication](https://en.wikipedia.org/wiki/Reduplication)

qoqo is a toolkit to represent quantum circuits by [HQS Quantum Simulations](https://quantumsimulations.de).

The original qoqo repository contains two components:

* roqoqo: the core rust library
* qoqo: the python interface to roqoqo

The examples provided here are written in Rust. These are the examples for **roqoqo** - the core rust library.

What roqoqo/qoqo is:

* A toolkit to represent quantum operations and circuits
* A thin runtime to run quantum measurements
* A way to serialize quantum circuits and measurement information
* A set of optional interfaces to devices, simulators and toolkits (e.g. [qoqo_qest](https://github.com/HQSquantumsimulations/qoqo-quest), [qoqo_mock](https://github.com/HQSquantumsimulations/qoqo_mock), [qoqo_qasm](https://github.com/HQSquantumsimulations/qoqo_qasm))

What roqoqo/qoqo is **not**:

* A decomposer translating circuits to a specific set of gates
* A quantum circuit optimizer
* A collection of quantum algorithms

## 1. A simple circuit and measurement

We show the construction of a simple entangling circuit and an observable measurement based on this circuit.

### 1.1 Entangling circuit snippet
Similar to many other toolkits the unitary entangling circuit can be constructed by adding operations to a circuit

In [3]:
:dep roqoqo = "0.10.0"
extern crate roqoqo;
use roqoqo::{Circuit, operations::*};
use std::collections::HashSet;

// Create a new modifiable circuit
let mut circuit = Circuit::new();
// Prepare qubits 0 and 1 in a superposition state by adding the Hadamard gate
circuit += Hadamard::new(0);
circuit += Hadamard::new(1);
// Establish entanglement between qubits 0 and 1
circuit += CNOT::new(0,1);

// Print
println!("{:?}", circuit);
println!("{} {:?}", "Circuit length: ", circuit.len());
println!("{} {:?}", "Operation types: ", circuit.get_operation_types());

// For demonstrative purposes only:
// Compare obtained operation types to those expected for this example
let mut types: HashSet<&str> = HashSet::new();
types.insert("CNOT");
types.insert("Hadamard");
assert_eq!(circuit.get_operation_types(), types);

// Compare the derived circuit length to the expected one for this example
assert_eq!(circuit.len(), 3);

Circuit { definitions: [], operations: [Hadamard(Hadamard { qubit: 0 }), Hadamard(Hadamard { qubit: 1 }), CNOT(CNOT { control: 0, target: 1 })], _roqoqo_version: RoqoqoVersion }
Circuit length:  3
Operation types:  {"CNOT", "Hadamard"}


### 1.2 Measuring qubits
Qoqo uses classical registers for the readout. We need to add a classical register definition to the circuit and a measurement statement.
The number of projective measurements can be directly set in the circuit.  
The simulation and measurement of the circuit is handled by the roqoqo_quest interface (in this example).

In [4]:
:dep roqoqo = "0.10.0"
:dep roqoqo-quest = "0.3.0"

extern crate roqoqo;
extern crate roqoqo_quest;
use roqoqo_quest::Backend;
use roqoqo::{Circuit, operations::*, registers::*};
use roqoqo::backends::{EvaluatingBackend, RegisterResult};
use std::collections::HashMap;

// Create new modifiable circuit
let mut circuit = Circuit::new();
// Define classical bit register for the readout of the measurement
let register_name: String = "ro".to_string();
circuit += DefinitionBit::new(register_name.clone(), 2, true);
// Add operations to the circuit
circuit += Hadamard::new(0);
circuit += CNOT::new(0,1);
// Add operation to the circuit to perform repeated measurements in a quantum computing simulation.
circuit += PragmaRepeatedMeasurement::new(register_name.clone(), 10, None);
println!("{} {:?}", ">> Circuit prepared for a simulated measurement: ", circuit);

let backend = Backend::new(2);
let result: RegisterResult = backend.run_circuit(&circuit);
let result_registers: (
    HashMap<String, BitOutputRegister>,
    HashMap<String, FloatOutputRegister>,
    HashMap<String, ComplexOutputRegister>,
) = result.unwrap();

println!(">> Bit output register 'ro' contains the following single projective measurements:");
for single_projective_measurements in &result_registers.0["ro"] {
    println!("{:?}", single_projective_measurements);
};

>> Circuit prepared for a simulated measurement:  Circuit { definitions: [DefinitionBit(DefinitionBit { name: "ro", length: 2, is_output: true })], operations: [Hadamard(Hadamard { qubit: 0 }), CNOT(CNOT { control: 0, target: 1 }), PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro", number_measurements: 10, qubit_mapping: None })], _roqoqo_version: RoqoqoVersion }
>> Bit output register 'ro' contains the following single projective measurements:
[false, false]
[false, false]
[true, true]
[false, false]
[false, false]
[true, true]
[true, true]
[false, false]
[false, false]
[true, true]


### 1.3 Measuring Observables
Qoqo includes the direct evaluation of projective measurements to an observable measurement e.g. 3 * < Z0 > + < Z0 Z1 >.
The measurement is defined by a set of expectation values of a product of pauli operators and a matrix that combines the expectation values.

In [5]:
:dep roqoqo = "0.10.0"
:dep roqoqo-quest = "0.3.0"

extern crate roqoqo;
extern crate roqoqo_quest;
use roqoqo_quest::Backend;
use roqoqo::{Circuit, operations::*, registers::*, QuantumProgram};
use roqoqo::measurements::{BasisRotationInput, BasisRotation};
use std::collections::HashMap;

let mut circuit = Circuit::new();
circuit.add_operation(DefinitionBit::new("ro".to_string(), 2, true));
circuit.add_operation(Hadamard::new(0));
circuit.add_operation(CNOT::new(0,1));
circuit.add_operation(PragmaRepeatedMeasurement::new("ro".to_string(), 10, None));

let mut measurement_input = BasisRotationInput::new(2, false);
// From readout 'ro' measure two pauli products 0: < Z0 > and 1: < Z0 Z1 >
measurement_input.add_pauli_product("ro".to_string(), vec![0]);
measurement_input.add_pauli_product("ro".to_string(), vec![0, 1]);
// One expectation value: 3 * pauli_product0 + 1 * pauli_product1
measurement_input.add_linear_exp_val("example".to_string(), HashMap::from([(0, 3.0), (1, 1.0)]));
println!("{} {:?}", ">> Measurement input defined: ", measurement_input);

let measurement = BasisRotation {
    input: measurement_input,
    circuits: vec![circuit.clone()],
    constant_circuit: None,
};
let backend = Backend::new(2);
let program = QuantumProgram::BasisRotation {
    measurement: measurement,
    input_parameter_names: vec![],
};

// Measurement result
let result = program.run(backend, &[]).unwrap().unwrap()["example"];
println!("{} {:?}", ">> Result of Quantum Program: ", result);

// Validation check
assert!(result > - 4.0 * 10.0);
assert!(result < 4.0 * 10.0);


>> Measurement input defined:  BasisRotationInput { pauli_product_qubit_masks: {"ro": {0: [0], 1: [0, 1]}}, number_qubits: 2, number_pauli_products: 2, measured_exp_vals: {"example": Linear({1: 1.0, 0: 3.0})}, use_flipped_measurement: false }
>> Result of Quantum Program:  0.3999999999999999


### 1.4 De/Serializing the quantum program

Same procedure as introduced in the example 1.3 "Measurement observables", but now the measurement, and afterwards the quantum program, are serialized to and de-serialized from json. The measurement result is compared before and after the de/-serialization.

In [6]:
:dep roqoqo = "0.10.0"
:dep roqoqo-quest = "0.3.0"

extern crate roqoqo;
extern crate roqoqo_quest;
use roqoqo_quest::Backend;
extern crate serde_json;
use roqoqo::{Circuit, operations::*, registers::*, QuantumProgram};
use roqoqo::measurements::{BasisRotationInput, BasisRotation};

use std::collections::HashMap;

let mut circuit = Circuit::new();
circuit.add_operation(DefinitionBit::new("ro".to_string(), 2, true));
circuit.add_operation(PauliX::new(0));
circuit.add_operation(CNOT::new(0,1));
circuit.add_operation(PragmaRepeatedMeasurement::new("ro".to_string(), 10, None));

let mut measurement_input = BasisRotationInput::new(2, false);
// From readout 'ro' measure two pauli products 0: < Z0 > and 1: < Z0 Z1 >
measurement_input.add_pauli_product("ro".to_string(), vec![0]);
measurement_input.add_pauli_product("ro".to_string(), vec![0, 1]);
// One expectation value: 3 * pauli_product0 + 1 * pauli_product1
measurement_input.add_linear_exp_val("example".to_string(), HashMap::from([(0, 3.0), (1, 1.0)]));
println!("{} {:?}", ">> Measurement input defined: ", measurement_input);

let measurement = BasisRotation {
    input: measurement_input,
    circuits: vec![circuit.clone()],
    constant_circuit: None,
};
let backend = Backend::new(2);
let program = QuantumProgram::BasisRotation {
    measurement: measurement.clone(),
    input_parameter_names: vec![],
};

// First, let's de-/serialize the BasisRotation measurement and test the outcome
let measurement_json = serde_json::to_string(&measurement).unwrap();
let measurement_new: BasisRotation = serde_json::from_str(&measurement_json).unwrap();
assert!(measurement_new == measurement);
println!(">> De/Serialization of BasisRotation performed successfully.");

// Next, de-/serialize the Quantum Program
let program_json = serde_json::to_string(&program).unwrap();
let program_new: QuantumProgram = serde_json::from_str(&program_json).unwrap();
assert!(program == program_new.clone());
println!(">> De/Serialization of QuantumProgram performed successfully.");

>> Measurement input defined:  BasisRotationInput { pauli_product_qubit_masks: {"ro": {1: [0, 1], 0: [0]}}, number_qubits: 2, number_pauli_products: 2, measured_exp_vals: {"example": Linear({0: 3.0, 1: 1.0})}, use_flipped_measurement: false }
>> De/Serialization of BasisRotation performed successfully.
>> De/Serialization of QuantumProgram performed successfully.


## 2. Fine control over decoherence
Qoqo allows full control over decoherence by placing decoherence operations in the circuit on the same level as gates.  
Example: Letting only one qubit decay.  
The backend automatically switches from statevector simulation to density matrix simulation in the presence of noise.

In [7]:
:dep roqoqo = "0.10.0"
:dep roqoqo-quest = "0.3.0"

extern crate roqoqo;
extern crate roqoqo_quest;
use roqoqo_quest::Backend;
use roqoqo::{Circuit, operations::*, registers::*};
use roqoqo::backends::{EvaluatingBackend, RegisterResult};
use std::collections::HashMap;

let damping = 0.1;
let number_measurements = 100;
let mut circuit = Circuit::new();
circuit += DefinitionBit::new("ro".to_string(), 2, true);
circuit += PauliX::new(0);
circuit += PauliX::new(1);
circuit += PragmaDamping::new(0, 1.into(), damping.into());
circuit += PragmaRepeatedMeasurement::new("ro".to_string(), number_measurements, None);
println!("{} {:?}", ">> Circuit: ", circuit);

let backend = Backend::new(2);
let result: RegisterResult = backend.run_circuit(&circuit);
println!("{} {:?}", ">> Raw result: ", result);
let result_registers: (
    HashMap<String, BitOutputRegister>,
    HashMap<String, FloatOutputRegister>,
    HashMap<String, ComplexOutputRegister>,
) = result.unwrap();

let mut sum_test = [0.0, 0.0];
for single_measurement in &result_registers.0["ro"] {
    for i in 0..2 {
        let single_projective_measurement = match single_measurement[i] {
            true => 1.0,
            false => 0.0,
        };
        sum_test[i] += single_projective_measurement
    }
};
println!("{:?}", sum_test);
let scaled_result = [sum_test[0] / number_measurements as f64, sum_test[1] / number_measurements as f64];
 println!("{} {:?}", ">> Scaled result: ", scaled_result);

>> Circuit:  Circuit { definitions: [DefinitionBit(DefinitionBit { name: "ro", length: 2, is_output: true })], operations: [PauliX(PauliX { qubit: 0 }), PauliX(PauliX { qubit: 1 }), PragmaDamping(PragmaDamping { qubit: 0, gate_time: Float(1.0), rate: Float(0.1) }), PragmaRepeatedMeasurement(PragmaRepeatedMeasurement { readout: "ro", number_measurements: 100, qubit_mapping: None })], _roqoqo_version: RoqoqoVersion }
>> Raw result:  Ok(({"ro": [[true, true], [false, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [false, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true, true], [true

## 3. Symbolic parameters
In many cases, operation parameters depend on a symbolic parameter of the whole quantum program (time in time-evolution, overrotation, variational parameters...)  
Qoqo allows the fast calculation of symbolic parameter expressions.  
Expressions are provided in string form.  
DoUnitary can automatically replace symbolic parameters using call parameters.

### 3.1 Writing the symbolic circuit and replacing symbolic parameters

In [8]:
:dep roqoqo = "0.10.0"
:dep qoqo_calculator = "0.6"

extern crate roqoqo;
extern crate qoqo_calculator;
use roqoqo::{Circuit, operations::*};
use qoqo_calculator::Calculator;

let mut circuit = Circuit::new();
circuit.add_operation(RotateX::new(0, "3*time+offset".into()));
println!("{} {:?}", "Symbolic circuit: ", circuit);

let mut substitution_dict: Calculator = Calculator::new();
substitution_dict.set_variable("time", 1.0 / 3.0);
substitution_dict.set_variable("offset", 1.0);
let circuit2 = circuit.substitute_parameters(&mut substitution_dict).unwrap();
println!("{} {:?}", "After substitution: ", circuit2);

Symbolic circuit:  Circuit { definitions: [], operations: [RotateX(RotateX { qubit: 0, theta: Str("3*time+offset") })], _roqoqo_version: RoqoqoVersion }
After substitution:  Circuit { definitions: [], operations: [RotateX(RotateX { qubit: 0, theta: Float(2.0) })], _roqoqo_version: RoqoqoVersion }


### 3.2 Symbolic parameters in a full quantum program

In [9]:
:dep roqoqo = "0.10.0"
:dep roqoqo-quest = "0.3.0"
:dep qoqo_calculator = "0.6"

extern crate roqoqo;
extern crate roqoqo_quest;
extern crate qoqo_calculator;
use qoqo_calculator::Calculator;
use roqoqo_quest::Backend;
use roqoqo::{Circuit, operations::*, registers::*, QuantumProgram};
use roqoqo::measurements::{BasisRotationInput, BasisRotation};
use std::collections::HashMap;

let number_measurements = 100000;

let mut circuit = Circuit::new();
circuit.add_operation(DefinitionBit::new("ro".to_string(), 2, true));
circuit.add_operation(RotateX::new(0, "3*time+offset".into()));
circuit.add_operation(PragmaRepeatedMeasurement::new("ro".to_string(), 10, None));

let mut measurement_input = BasisRotationInput::new(2, false);
measurement_input.add_pauli_product("ro".to_string(), vec![0]);
measurement_input.add_pauli_product("ro".to_string(), vec![0, 1]);
measurement_input.add_linear_exp_val("example".to_string(), HashMap::from([(0, 3.0), (1, 1.0)]));

let measurement = BasisRotation {
    input: measurement_input,
    circuits: vec![circuit.clone()],
    constant_circuit: None,
};
let backend = Backend::new(2);
// The symbolic parameters are passed to the QuantumProgram. These are the free parameters.
let program = QuantumProgram::BasisRotation {
    measurement: measurement,
    input_parameter_names: vec!["time".to_string(), "offset".to_string()],
};

// Set the free parameters when executing the program
let result = program.run(backend, &[0.5, 0.0]).unwrap().unwrap()["example"];
println!("{} {:?}", "Result of Quantum Program: ", result);

Result of Quantum Program:  0.8


## 4. Testing scaling performance with qoqo_mock
Quantum simulators cannot simulate systems with a significant number of qubits fast enough to benchmark qoqo with a large number of qubits and operations.
The qoqo_mock interface can be used to benchmark qoqo without simulating a quantum computer.

In [13]:
:dep rand = "0.8.4"
:dep roqoqo = "0.9.0"
:dep roqoqo-mock = "0.2.2"

extern crate roqoqo;
extern crate roqoqo_mock;
extern crate timeit;

use roqoqo::{Circuit, operations::*, registers::*, QuantumProgram};
use roqoqo::measurements::{BasisRotationInput, BasisRotation};
use roqoqo_mock::Backend;
use timeit::*;

use rand::distributions::{Standard, Uniform};
use rand::prelude::*;
use rand::rngs::StdRng;
use rand::thread_rng;
use std::collections::HashMap;

let number_measurements = 10;
let number_operations = 100;
let number_qubits: usize = 5;

let mut circuit = Circuit::new();
circuit.add_operation(DefinitionBit::new("ro".to_string(), number_qubits, true));

let mut rng = thread_rng();
let qubits_distribution = Uniform::from(0..number_qubits);
let two_qubits_distribution = Uniform::from(0..number_qubits-1);
let operations_distribution = Uniform::from(0..number_operations);

let new_op: Operation = match operations_distribution.sample(&mut rng) {
    0 => RotateX::new(qubits_distribution.sample(&mut rng), "4*theta_x".to_string().into()).into(),
    1 => RotateY::new(qubits_distribution.sample(&mut rng), "2*theta_y".to_string().into()).into(),
    2 => RotateZ::new(qubits_distribution.sample(&mut rng), "3*theta_z".to_string().into()).into(),
    4 => {
        let qubit = two_qubits_distribution.sample(&mut rng);
        ControlledPauliZ::new(qubit, qubit+1).into()
    }
    _ => RotateZ::new(qubits_distribution.sample(&mut rng), 0.into()).into(),
};

circuit.add_operation(new_op);
circuit.add_operation(PragmaRepeatedMeasurement::new("ro".to_string(), number_measurements, None));

let mut measurement_input = BasisRotationInput::new(number_qubits, false);
for i in 0..number_qubits-1 {
    measurement_input.add_pauli_product("ro".to_string(), vec![i]);
}
measurement_input.add_linear_exp_val("example".to_string(), HashMap::from([(0, 1.0)]));
let measurement = BasisRotation {
    input: measurement_input,
    circuits: vec![circuit.clone()],
    constant_circuit: None,
};
let backend = Backend::new(number_qubits);

let program = QuantumProgram::BasisRotation {
    measurement: measurement,
    input_parameter_names: vec!["theta_x".to_string(), "theta_y".to_string(), "theta_z".to_string()],
};

let sec = timeit_loops!(1, {
    let result = program.run(backend.clone(), &[0.0, 1.0, 2.0]).unwrap().unwrap()["example"];
    println!("{} {:?}", "Result of Quantum Program: ", result);
});
println!("{} {:?}", "Program run time [in sec]: ", sec);

get_time();


The type of the variable measurement_new was redefined, so was lost.
The type of the variable program_new was redefined, so was lost.
The type of the variable circuit was redefined, so was lost.
The type of the variable program was redefined, so was lost.
The type of the variable circuit2 was redefined, so was lost.


Result of Quantum Program:  0.0
Program run time [in sec]:  3.2846e-5
